
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# 6 - Ingesting JSON Files with Databricks

In this demonstration, we’ll explore how to ingest JSON files and perform foundational JSON-specific transformations during ingestion, including decoding encoded fields and flattening nested JSON strings. We’ll be working with simulated Kafka event data, sourced from Databricks Marketplace.

### Learning Objectives
By the end of this lesson, you should be able to:
- Ingest raw JSON data into Unity Catalog using CTAS and `read_files()`.
- Apply multiple techniques to flatten JSON string columns with and without converting to a STRUCT type.
- Understand the difference between `explode()` and `explode_outer()`.
- Introduce the capabilities and use cases of the VARIANT data type (public preview as of Q2-2025)

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default and you have a Shared SQL warehouse.

<!-- ![Select Cluster](./Includes/images/selecting_cluster_info.png) -->

Follow these steps to select the classic compute cluster:


1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.

   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.


## A. Classroom Setup

Run the following cell to configure your working environment for this notebook.

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course in the lab environment.

In [0]:
%run ./Includes/Classroom-Setup-06

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


----------------------------------------------------------------------------------------
Creating folder: /Volumes/dbacademy/ops/labuser10983516_1758981184@vocareum_com/csv_demo_files
Creating folder: /Volumes/dbacademy/ops/labuser10983516_1758981184@vocareum_com/json_demo_files
Creating folder: /Volumes/dbacademy/ops/labuser10983516_1758981184@vocareum_com/xml_demo_files
----------------------------------------------------------------------------------------



Run the cell below to view your default catalog and schema. Notice that your default catalog is **dbacademy** and your default schema is your unique **labuser** schema.

**NOTE:** The default catalog and schema are pre-configured for you to avoid the need to specify the three-level name when writing your tables (i.e., catalog.schema.table).

In [0]:
SELECT current_catalog(), current_schema()

current_catalog(),current_schema()
dbacademy,labuser10983516_1758981184


## B. Overview of CTAS with `read_files()` for Ingestion of JSON files

### B1. Inspect JSON files

1. Run the next cell to verify that there are 11 JSON files located at `/Volumes/dbacademy_ecommerce/v01/raw/events-kafka`.

In [0]:
LIST '/Volumes/dbacademy_ecommerce/v01/raw/events-kafka'

path,name,size,modification_time
/Volumes/dbacademy_ecommerce/v01/raw/events-kafka/000.json,000.json,200116,1726173033000
/Volumes/dbacademy_ecommerce/v01/raw/events-kafka/001.json,001.json,169907,1726173033000
/Volumes/dbacademy_ecommerce/v01/raw/events-kafka/002.json,002.json,140680,1726173033000
/Volumes/dbacademy_ecommerce/v01/raw/events-kafka/003.json,003.json,139280,1726173033000
/Volumes/dbacademy_ecommerce/v01/raw/events-kafka/004.json,004.json,122411,1726173034000
/Volumes/dbacademy_ecommerce/v01/raw/events-kafka/005.json,005.json,96034,1726173034000
/Volumes/dbacademy_ecommerce/v01/raw/events-kafka/006.json,006.json,98332,1726173034000
/Volumes/dbacademy_ecommerce/v01/raw/events-kafka/007.json,007.json,86452,1726173034000
/Volumes/dbacademy_ecommerce/v01/raw/events-kafka/008.json,008.json,68052,1726173034000
/Volumes/dbacademy_ecommerce/v01/raw/events-kafka/009.json,009.json,44729,1726173035000


2. Run the cell below to view the raw JSON data in the output. Note the following:

   - Each row contains JSON with 6 key/value pairs.

   - The **key** and **value** fields are encoded in base64. Base64 is an encoding scheme that converts binary data into a readable ASCII string.



<br></br>
**Example Output Formatted**
```
{
    "key": "VUEwMDAwMDAxMDczOTgwNTQ=",
    "offset": 219255030,
    "partition": 0,
    "timestamp": 1593880885085,
    "topic": "clickstream",
    "value": "eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6Im1haW4iLCJldmVudF90aW1lc3R
    hbXAiOjE1OTM4ODA4ODUwMzYxMjksImdlbyI6eyJjaXR5IjoiTmV3IFlvcmsiLCJzdGF0ZSI6Ik5ZIn0sIml0ZW1zIjp
    bXSwidHJhZmZpY19zb3VyY2UiOiJnb29nbGUiLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg4MDg4NTA
    zNjEyOSwidXNlcl9pZCI6IlVBMDAwMDAwMTA3Mzk4MDU0In0=",
}
```

In [0]:
SELECT * 
FROM text.`/Volumes/dbacademy_ecommerce/v01/raw/events-kafka`
LIMIT 5;

value
"{""key"":""VUEwMDAwMDAxMDczOTgwNTQ="",""offset"":219255030,""partition"":0,""timestamp"":1593880885085,""topic"":""clickstream"",""value"":""eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6Im1haW4iLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODUwMzYxMjksImdlbyI6eyJjaXR5IjoiTmV3IFlvcmsiLCJzdGF0ZSI6Ik5ZIn0sIml0ZW1zIjpbXSwidHJhZmZpY19zb3VyY2UiOiJnb29nbGUiLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg4MDg4NTAzNjEyOSwidXNlcl9pZCI6IlVBMDAwMDAwMTA3Mzk4MDU0In0=""}"
"{""key"":""VUEwMDAwMDAxMDczOTI0NTg="",""offset"":219255043,""partition"":0,""timestamp"":1593880892303,""topic"":""clickstream"",""value"":""eyJkZXZpY2UiOiJpT1MiLCJlY29tbWVyY2UiOnt9LCJldmVudF9uYW1lIjoiYWRkX2l0ZW0iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzMDA2OTY3NTEsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg5MjI1MTMxMCwiZ2VvIjp7ImNpdHkiOiJXZXN0YnJvb2siLCJzdGF0ZSI6Ik1FIn0sIml0ZW1zIjpbeyJpdGVtX2lkIjoiTV9TVEFOX1QiLCJpdGVtX25hbWUiOiJTdGFuZGFyZCBUd2luIE1hdHRyZXNzIiwiaXRlbV9yZXZlbnVlX2luX3VzZCI6NTk1LjAsInByaWNlX2luX3VzZCI6NTk1LjAsInF1YW50aXR5IjoxfV0sInRyYWZmaWNfc291cmNlIjoiZ29vZ2xlIiwidXNlcl9maXJzdF90b3VjaF90aW1lc3RhbXAiOjE1OTM4ODAzMDA2OTY3NTEsInVzZXJfaWQiOiJVQTAwMDAwMDEwNzM5MjQ1OCJ9""}"
"{""key"":""VUEwMDAwMDAxMDczOTU5Njg="",""offset"":219255108,""partition"":0,""timestamp"":1593880889174,""topic"":""clickstream"",""value"":""eyJkZXZpY2UiOiJtYWNPUyIsImVjb21tZXJjZSI6e30sImV2ZW50X25hbWUiOiJwcmVtaXVtIiwiZXZlbnRfcHJldmlvdXNfdGltZXN0YW1wIjoxNTkzODgwODYxMDMwMjQxLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODkxMjY3NzgsImdlbyI6eyJjaXR5IjoiRmlzaGVycyIsInN0YXRlIjoiSU4ifSwiaXRlbXMiOltdLCJ0cmFmZmljX3NvdXJjZSI6InlvdXR1YmUiLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg4MDY2NDY1ODc3MywidXNlcl9pZCI6IlVBMDAwMDAwMTA3Mzk1OTY4In0=""}"
"{""key"":""VUEwMDAwMDAxMDczOTgwMzA="",""offset"":219255118,""partition"":0,""timestamp"":1593880889725,""topic"":""clickstream"",""value"":""eyJkZXZpY2UiOiJpT1MiLCJlY29tbWVyY2UiOnt9LCJldmVudF9uYW1lIjoib3JpZ2luYWwiLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODA4ODI0Mjk5ODAsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg4OTY3Njg1NywiZ2VvIjp7ImNpdHkiOiJMb21pdGEiLCJzdGF0ZSI6IkNBIn0sIml0ZW1zIjpbXSwidHJhZmZpY19zb3VyY2UiOiJmYWNlYm9vayIsInVzZXJfZmlyc3RfdG91Y2hfdGltZXN0YW1wIjoxNTkzODgwODgyNDI5OTgwLCJ1c2VyX2lkIjoiVUEwMDAwMDAxMDczOTgwMzAifQ==""}"
"{""key"":""VUEwMDAwMDAxMDczODIyMzM="",""offset"":219438025,""partition"":1,""timestamp"":1593880886106,""topic"":""clickstream"",""value"":""eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6ImNjX2luZm8iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzNjQzMjEwODgsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg4NjA2NTEyNSwiZ2VvIjp7ImNpdHkiOiJOZXcgWW9yayIsInN0YXRlIjoiTlkifSwiaXRlbXMiOlt7Iml0ZW1faWQiOiJNX1NUQU5fRiIsIml0ZW1fbmFtZSI6IlN0YW5kYXJkIEZ1bGwgTWF0dHJlc3MiLCJpdGVtX3JldmVudWVfaW5fdXNkIjo5NDUuMCwicHJpY2VfaW5fdXNkIjo5NDUuMCwicXVhbnRpdHkiOjF9XSwidHJhZmZpY19zb3VyY2UiOiJpbnN0YWdyYW0iLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg3OTE5NTI2MjA0NywidXNlcl9pZCI6IlVBMDAwMDAwMTA3MzgyMjMzIn0=""}"


3. Run the cell below to see how to use `read_files()` to read the JSON data. Notice the following:

   - The JSON file is cleanly read into a tabular format with 6 columns.

   - The **key** and **value** columns are base64-encoded and returned as STRING data type.
   
   - There are no rows in the **_rescued_data** column.

In [0]:
SELECT *
FROM read_files(
  "/Volumes/dbacademy_ecommerce/v01/raw/events-kafka",
  format => "json"
)
LIMIT 10;

key,offset,partition,timestamp,topic,value,_rescued_data
VUEwMDAwMDAxMDczOTgwNTQ=,219255030,0,1593880885085,clickstream,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6Im1haW4iLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODUwMzYxMjksImdlbyI6eyJjaXR5IjoiTmV3IFlvcmsiLCJzdGF0ZSI6Ik5ZIn0sIml0ZW1zIjpbXSwidHJhZmZpY19zb3VyY2UiOiJnb29nbGUiLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg4MDg4NTAzNjEyOSwidXNlcl9pZCI6IlVBMDAwMDAwMTA3Mzk4MDU0In0=,null
VUEwMDAwMDAxMDczOTI0NTg=,219255043,0,1593880892303,clickstream,eyJkZXZpY2UiOiJpT1MiLCJlY29tbWVyY2UiOnt9LCJldmVudF9uYW1lIjoiYWRkX2l0ZW0iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzMDA2OTY3NTEsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg5MjI1MTMxMCwiZ2VvIjp7ImNpdHkiOiJXZXN0YnJvb2siLCJzdGF0ZSI6Ik1FIn0sIml0ZW1zIjpbeyJpdGVtX2lkIjoiTV9TVEFOX1QiLCJpdGVtX25hbWUiOiJTdGFuZGFyZCBUd2luIE1hdHRyZXNzIiwiaXRlbV9yZXZlbnVlX2luX3VzZCI6NTk1LjAsInByaWNlX2luX3VzZCI6NTk1LjAsInF1YW50aXR5IjoxfV0sInRyYWZmaWNfc291cmNlIjoiZ29vZ2xlIiwidXNlcl9maXJzdF90b3VjaF90aW1lc3RhbXAiOjE1OTM4ODAzMDA2OTY3NTEsInVzZXJfaWQiOiJVQTAwMDAwMDEwNzM5MjQ1OCJ9,null
VUEwMDAwMDAxMDczOTU5Njg=,219255108,0,1593880889174,clickstream,eyJkZXZpY2UiOiJtYWNPUyIsImVjb21tZXJjZSI6e30sImV2ZW50X25hbWUiOiJwcmVtaXVtIiwiZXZlbnRfcHJldmlvdXNfdGltZXN0YW1wIjoxNTkzODgwODYxMDMwMjQxLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODkxMjY3NzgsImdlbyI6eyJjaXR5IjoiRmlzaGVycyIsInN0YXRlIjoiSU4ifSwiaXRlbXMiOltdLCJ0cmFmZmljX3NvdXJjZSI6InlvdXR1YmUiLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg4MDY2NDY1ODc3MywidXNlcl9pZCI6IlVBMDAwMDAwMTA3Mzk1OTY4In0=,null
VUEwMDAwMDAxMDczOTgwMzA=,219255118,0,1593880889725,clickstream,eyJkZXZpY2UiOiJpT1MiLCJlY29tbWVyY2UiOnt9LCJldmVudF9uYW1lIjoib3JpZ2luYWwiLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODA4ODI0Mjk5ODAsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg4OTY3Njg1NywiZ2VvIjp7ImNpdHkiOiJMb21pdGEiLCJzdGF0ZSI6IkNBIn0sIml0ZW1zIjpbXSwidHJhZmZpY19zb3VyY2UiOiJmYWNlYm9vayIsInVzZXJfZmlyc3RfdG91Y2hfdGltZXN0YW1wIjoxNTkzODgwODgyNDI5OTgwLCJ1c2VyX2lkIjoiVUEwMDAwMDAxMDczOTgwMzAifQ==,null
VUEwMDAwMDAxMDczODIyMzM=,219438025,1,1593880886106,clickstream,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6ImNjX2luZm8iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzNjQzMjEwODgsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg4NjA2NTEyNSwiZ2VvIjp7ImNpdHkiOiJOZXcgWW9yayIsInN0YXRlIjoiTlkifSwiaXRlbXMiOlt7Iml0ZW1faWQiOiJNX1NUQU5fRiIsIml0ZW1fbmFtZSI6IlN0YW5kYXJkIEZ1bGwgTWF0dHJlc3MiLCJpdGVtX3JldmVudWVfaW5fdXNkIjo5NDUuMCwicHJpY2VfaW5fdXNkIjo5NDUuMCwicXVhbnRpdHkiOjF9XSwidHJhZmZpY19zb3VyY2UiOiJpbnN0YWdyYW0iLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg3OTE5NTI2MjA0NywidXNlcl9pZCI6IlVBMDAwMDAwMTA3MzgyMjMzIn0=,null
VUEwMDAwMDAxMDczODIyMzM=,219438069,1,1593880886106,clickstream,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6ImNjX2luZm8iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzNjQzMjEwODgsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg4NjA2NTEyNSwiZ2VvIjp7ImNpdHkiOiJOZXcgWW9yayIsInN0YXRlIjoiTlkifSwiaXRlbXMiOlt7Iml0ZW1faWQiOiJNX1NUQU5fRiIsIml0ZW1fbmFtZSI6IlN0YW5kYXJkIEZ1bGwgTWF0dHJlc3MiLCJpdGVtX3JldmVudWVfaW5fdXNkIjo5NDUuMCwicHJpY2VfaW5fdXNkIjo5NDUuMCwicXVhbnRpdHkiOjF9XSwidHJhZmZpY19zb3VyY2UiOiJpbnN0YWdyYW0iLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg3OTE5NTI2MjA0NywidXNlcl9pZCI6IlVBMDAwMDAwMTA3MzgyMjMzIn0=,null
VUEwMDAwMDAxMDczOTgwMzc=,219438089,1,1593880887640,clickstream,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6ImRlbGl2ZXJ5IiwiZXZlbnRfcHJldmlvdXNfdGltZXN0YW1wIjoxNTkzODgwODgyOTY0MjYyLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODc2MDUzMzcsImdlbyI6eyJjaXR5IjoiVmVybm9uIiwic3RhdGUiOiJUWCJ9LCJpdGVtcyI6W10sInRyYWZmaWNfc291cmNlIjoiZmFjZWJvb2siLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg4MDg4Mjk2NDI2MiwidXNlcl9pZCI6IlVBMDAwMDAwMTA3Mzk4MDM3In0=,null
VUEwMDAwMDAxMDczOTgxNTk=,219438114,1,1593880894803,clickstream,eyJkZXZpY2UiOiJtYWNPUyIsImVjb21tZXJjZSI6e30sImV2ZW50X25hbWUiOiJtYWluIiwiZXZlbnRfdGltZXN0YW1wIjoxNTkzODgwODk0Nzg5NTc5LCJnZW8iOnsiY2l0eSI6Ikxha2V3b29kIiwic3RhdGUiOiJDTyJ9LCJpdGVtcyI6W10sInRyYWZmaWNfc291cmNlIjoieW91dHViZSIsInVzZXJfZmlyc3RfdG91Y2hfdGltZXN0YW1wIjoxNTkzODgwO

### B2. Using CTAS and `read_files()` with JSON

Ingesting JSON files using `read_files()` is as straightforward as reading CSV files.

1. Run the cell below to store this raw data in the **kafka_events_bronze_raw** table and view the table. When inspecting the results, you'll notice that:

   - The **key** and **value** columns are of type STRING and contain data that is **base64-encoded**.

   - This means the actual content has been encoded into base64 format and stored as a string. 
   
   - They have not yet been transformed into a readable string in the first bronze table we create.

**NOTE:** Base64 encoding is commonly used when ingesting data from sources like message queues or streaming platforms, where preserving formatting and avoiding data corruption is important.

In [0]:
-- Drop the table if it exists for demonstration purposes
DROP TABLE IF EXISTS kafka_events_bronze_raw;


-- Create the Delta table
CREATE TABLE kafka_events_bronze_raw AS
SELECT *
FROM read_files(
  "/Volumes/dbacademy_ecommerce/v01/raw/events-kafka",
  format => "json"
);


-- Display the table
SELECT *
FROM kafka_events_bronze_raw
LIMIT 10;

key,offset,partition,timestamp,topic,value,_rescued_data
VUEwMDAwMDAxMDczOTgwNTQ=,219255030,0,1593880885085,clickstream,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6Im1haW4iLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODUwMzYxMjksImdlbyI6eyJjaXR5IjoiTmV3IFlvcmsiLCJzdGF0ZSI6Ik5ZIn0sIml0ZW1zIjpbXSwidHJhZmZpY19zb3VyY2UiOiJnb29nbGUiLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg4MDg4NTAzNjEyOSwidXNlcl9pZCI6IlVBMDAwMDAwMTA3Mzk4MDU0In0=,null
VUEwMDAwMDAxMDczOTI0NTg=,219255043,0,1593880892303,clickstream,eyJkZXZpY2UiOiJpT1MiLCJlY29tbWVyY2UiOnt9LCJldmVudF9uYW1lIjoiYWRkX2l0ZW0iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzMDA2OTY3NTEsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg5MjI1MTMxMCwiZ2VvIjp7ImNpdHkiOiJXZXN0YnJvb2siLCJzdGF0ZSI6Ik1FIn0sIml0ZW1zIjpbeyJpdGVtX2lkIjoiTV9TVEFOX1QiLCJpdGVtX25hbWUiOiJTdGFuZGFyZCBUd2luIE1hdHRyZXNzIiwiaXRlbV9yZXZlbnVlX2luX3VzZCI6NTk1LjAsInByaWNlX2luX3VzZCI6NTk1LjAsInF1YW50aXR5IjoxfV0sInRyYWZmaWNfc291cmNlIjoiZ29vZ2xlIiwidXNlcl9maXJzdF90b3VjaF90aW1lc3RhbXAiOjE1OTM4ODAzMDA2OTY3NTEsInVzZXJfaWQiOiJVQTAwMDAwMDEwNzM5MjQ1OCJ9,null
VUEwMDAwMDAxMDczOTU5Njg=,219255108,0,1593880889174,clickstream,eyJkZXZpY2UiOiJtYWNPUyIsImVjb21tZXJjZSI6e30sImV2ZW50X25hbWUiOiJwcmVtaXVtIiwiZXZlbnRfcHJldmlvdXNfdGltZXN0YW1wIjoxNTkzODgwODYxMDMwMjQxLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODkxMjY3NzgsImdlbyI6eyJjaXR5IjoiRmlzaGVycyIsInN0YXRlIjoiSU4ifSwiaXRlbXMiOltdLCJ0cmFmZmljX3NvdXJjZSI6InlvdXR1YmUiLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg4MDY2NDY1ODc3MywidXNlcl9pZCI6IlVBMDAwMDAwMTA3Mzk1OTY4In0=,null
VUEwMDAwMDAxMDczOTgwMzA=,219255118,0,1593880889725,clickstream,eyJkZXZpY2UiOiJpT1MiLCJlY29tbWVyY2UiOnt9LCJldmVudF9uYW1lIjoib3JpZ2luYWwiLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODA4ODI0Mjk5ODAsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg4OTY3Njg1NywiZ2VvIjp7ImNpdHkiOiJMb21pdGEiLCJzdGF0ZSI6IkNBIn0sIml0ZW1zIjpbXSwidHJhZmZpY19zb3VyY2UiOiJmYWNlYm9vayIsInVzZXJfZmlyc3RfdG91Y2hfdGltZXN0YW1wIjoxNTkzODgwODgyNDI5OTgwLCJ1c2VyX2lkIjoiVUEwMDAwMDAxMDczOTgwMzAifQ==,null
VUEwMDAwMDAxMDczODIyMzM=,219438025,1,1593880886106,clickstream,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6ImNjX2luZm8iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzNjQzMjEwODgsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg4NjA2NTEyNSwiZ2VvIjp7ImNpdHkiOiJOZXcgWW9yayIsInN0YXRlIjoiTlkifSwiaXRlbXMiOlt7Iml0ZW1faWQiOiJNX1NUQU5fRiIsIml0ZW1fbmFtZSI6IlN0YW5kYXJkIEZ1bGwgTWF0dHJlc3MiLCJpdGVtX3JldmVudWVfaW5fdXNkIjo5NDUuMCwicHJpY2VfaW5fdXNkIjo5NDUuMCwicXVhbnRpdHkiOjF9XSwidHJhZmZpY19zb3VyY2UiOiJpbnN0YWdyYW0iLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg3OTE5NTI2MjA0NywidXNlcl9pZCI6IlVBMDAwMDAwMTA3MzgyMjMzIn0=,null
VUEwMDAwMDAxMDczODIyMzM=,219438069,1,1593880886106,clickstream,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6ImNjX2luZm8iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzNjQzMjEwODgsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg4NjA2NTEyNSwiZ2VvIjp7ImNpdHkiOiJOZXcgWW9yayIsInN0YXRlIjoiTlkifSwiaXRlbXMiOlt7Iml0ZW1faWQiOiJNX1NUQU5fRiIsIml0ZW1fbmFtZSI6IlN0YW5kYXJkIEZ1bGwgTWF0dHJlc3MiLCJpdGVtX3JldmVudWVfaW5fdXNkIjo5NDUuMCwicHJpY2VfaW5fdXNkIjo5NDUuMCwicXVhbnRpdHkiOjF9XSwidHJhZmZpY19zb3VyY2UiOiJpbnN0YWdyYW0iLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg3OTE5NTI2MjA0NywidXNlcl9pZCI6IlVBMDAwMDAwMTA3MzgyMjMzIn0=,null
VUEwMDAwMDAxMDczOTgwMzc=,219438089,1,1593880887640,clickstream,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6ImRlbGl2ZXJ5IiwiZXZlbnRfcHJldmlvdXNfdGltZXN0YW1wIjoxNTkzODgwODgyOTY0MjYyLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODc2MDUzMzcsImdlbyI6eyJjaXR5IjoiVmVybm9uIiwic3RhdGUiOiJUWCJ9LCJpdGVtcyI6W10sInRyYWZmaWNfc291cmNlIjoiZmFjZWJvb2siLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg4MDg4Mjk2NDI2MiwidXNlcl9pZCI6IlVBMDAwMDAwMTA3Mzk4MDM3In0=,null
VUEwMDAwMDAxMDczOTgxNTk=,219438114,1,1593880894803,clickstream,eyJkZXZpY2UiOiJtYWNPUyIsImVjb21tZXJjZSI6e30sImV2ZW50X25hbWUiOiJtYWluIiwiZXZlbnRfdGltZXN0YW1wIjoxNTkzODgwODk0Nzg5NTc5LCJnZW8iOnsiY2l0eSI6Ikxha2V3b29kIiwic3RhdGUiOiJDTyJ9LCJpdGVtcyI6W10sInRyYWZmaWNfc291cmNlIjoieW91dHViZSIsInVzZXJfZmlyc3RfdG91Y2hfdGltZXN0YW1wIjoxNTkzODgwO

### B3. Decoding base64 Strings for the Bronze Table

1. Let's take a look at decoding the **key** and **value** columns by inspecting their data types after applying the `unbase64()` function. The `unbase64` function returns a decoded base64 string as binary.

    - **encoded_key**: The original encoded **key** column as a base64 string.

    - **decoded_key**: A new column created by decoding **key** from a base64 string to BINARY.

    - **encoded_value**: The original encoded **value** column as a base64 string.

    - **decoded_value**: A new column created by decoding **value** from a base64 string to BINARY.

    Run the cell and view the results. Notice that the **decoded_key** and **decoded_value** columns are now BINARY.

In [0]:
SELECT
  key AS encoded_key,
  unbase64(key) AS decoded_key,
  value AS encoded_value,
  unbase64(value) AS decoded_value
FROM kafka_events_bronze_raw
LIMIT 5;

encoded_key,decoded_key,encoded_value,decoded_value
VUEwMDAwMDAxMDczOTgwNTQ=,VUEwMDAwMDAxMDczOTgwNTQ=,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6Im1haW4iLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODUwMzYxMjksImdlbyI6eyJjaXR5IjoiTmV3IFlvcmsiLCJzdGF0ZSI6Ik5ZIn0sIml0ZW1zIjpbXSwidHJhZmZpY19zb3VyY2UiOiJnb29nbGUiLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg4MDg4NTAzNjEyOSwidXNlcl9pZCI6IlVBMDAwMDAwMTA3Mzk4MDU0In0=,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6Im1haW4iLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODUwMzYxMjksImdlbyI6eyJjaXR5IjoiTmV3IFlvcmsiLCJzdGF0ZSI6Ik5ZIn0= (truncated)
VUEwMDAwMDAxMDczOTI0NTg=,VUEwMDAwMDAxMDczOTI0NTg=,eyJkZXZpY2UiOiJpT1MiLCJlY29tbWVyY2UiOnt9LCJldmVudF9uYW1lIjoiYWRkX2l0ZW0iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzMDA2OTY3NTEsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg5MjI1MTMxMCwiZ2VvIjp7ImNpdHkiOiJXZXN0YnJvb2siLCJzdGF0ZSI6Ik1FIn0sIml0ZW1zIjpbeyJpdGVtX2lkIjoiTV9TVEFOX1QiLCJpdGVtX25hbWUiOiJTdGFuZGFyZCBUd2luIE1hdHRyZXNzIiwiaXRlbV9yZXZlbnVlX2luX3VzZCI6NTk1LjAsInByaWNlX2luX3VzZCI6NTk1LjAsInF1YW50aXR5IjoxfV0sInRyYWZmaWNfc291cmNlIjoiZ29vZ2xlIiwidXNlcl9maXJzdF90b3VjaF90aW1lc3RhbXAiOjE1OTM4ODAzMDA2OTY3NTEsInVzZXJfaWQiOiJVQTAwMDAwMDEwNzM5MjQ1OCJ9,eyJkZXZpY2UiOiJpT1MiLCJlY29tbWVyY2UiOnt9LCJldmVudF9uYW1lIjoiYWRkX2l0ZW0iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzMDA2OTY3NTEsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg5MjI= (truncated)
VUEwMDAwMDAxMDczOTU5Njg=,VUEwMDAwMDAxMDczOTU5Njg=,eyJkZXZpY2UiOiJtYWNPUyIsImVjb21tZXJjZSI6e30sImV2ZW50X25hbWUiOiJwcmVtaXVtIiwiZXZlbnRfcHJldmlvdXNfdGltZXN0YW1wIjoxNTkzODgwODYxMDMwMjQxLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODkxMjY3NzgsImdlbyI6eyJjaXR5IjoiRmlzaGVycyIsInN0YXRlIjoiSU4ifSwiaXRlbXMiOltdLCJ0cmFmZmljX3NvdXJjZSI6InlvdXR1YmUiLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg4MDY2NDY1ODc3MywidXNlcl9pZCI6IlVBMDAwMDAwMTA3Mzk1OTY4In0=,eyJkZXZpY2UiOiJtYWNPUyIsImVjb21tZXJjZSI6e30sImV2ZW50X25hbWUiOiJwcmVtaXVtIiwiZXZlbnRfcHJldmlvdXNfdGltZXN0YW1wIjoxNTkzODgwODYxMDMwMjQxLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODk= (truncated)
VUEwMDAwMDAxMDczOTgwMzA=,VUEwMDAwMDAxMDczOTgwMzA=,eyJkZXZpY2UiOiJpT1MiLCJlY29tbWVyY2UiOnt9LCJldmVudF9uYW1lIjoib3JpZ2luYWwiLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODA4ODI0Mjk5ODAsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg4OTY3Njg1NywiZ2VvIjp7ImNpdHkiOiJMb21pdGEiLCJzdGF0ZSI6IkNBIn0sIml0ZW1zIjpbXSwidHJhZmZpY19zb3VyY2UiOiJmYWNlYm9vayIsInVzZXJfZmlyc3RfdG91Y2hfdGltZXN0YW1wIjoxNTkzODgwODgyNDI5OTgwLCJ1c2VyX2lkIjoiVUEwMDAwMDAxMDczOTgwMzAifQ==,eyJkZXZpY2UiOiJpT1MiLCJlY29tbWVyY2UiOnt9LCJldmVudF9uYW1lIjoib3JpZ2luYWwiLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODA4ODI0Mjk5ODAsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg4OTY= (truncated)
VUEwMDAwMDAxMDczODIyMzM=,VUEwMDAwMDAxMDczODIyMzM=,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6ImNjX2luZm8iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzNjQzMjEwODgsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg4NjA2NTEyNSwiZ2VvIjp7ImNpdHkiOiJOZXcgWW9yayIsInN0YXRlIjoiTlkifSwiaXRlbXMiOlt7Iml0ZW1faWQiOiJNX1NUQU5fRiIsIml0ZW1fbmFtZSI6IlN0YW5kYXJkIEZ1bGwgTWF0dHJlc3MiLCJpdGVtX3JldmVudWVfaW5fdXNkIjo5NDUuMCwicHJpY2VfaW5fdXNkIjo5NDUuMCwicXVhbnRpdHkiOjF9XSwidHJhZmZpY19zb3VyY2UiOiJpbnN0YWdyYW0iLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg3OTE5NTI2MjA0NywidXNlcl9pZCI6IlVBMDAwMDAwMTA3MzgyMjMzIn0=,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6ImNjX2luZm8iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzNjQzMjEwODgsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg= (truncated)


2. Run the next cell to convert the BINARY columns to STRING columns using the `CAST` function. Notice the following in the results:

    - The **decoded_key** and **decoded_value** columns are now of type STRING and readable.

    - The **decoded_value** column is a JSON-formatted string.


In [0]:
SELECT
  key AS encoded_key,
  cast(unbase64(key) AS STRING) AS decoded_key,
  value AS encoded_value,
  cast(unbase64(value) AS STRING) AS decoded_value
FROM kafka_events_bronze_raw
LIMIT 5;

encoded_key,decoded_key,encoded_value,decoded_value
VUEwMDAwMDAxMDczOTgwNTQ=,UA000000107398054,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6Im1haW4iLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODUwMzYxMjksImdlbyI6eyJjaXR5IjoiTmV3IFlvcmsiLCJzdGF0ZSI6Ik5ZIn0sIml0ZW1zIjpbXSwidHJhZmZpY19zb3VyY2UiOiJnb29nbGUiLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg4MDg4NTAzNjEyOSwidXNlcl9pZCI6IlVBMDAwMDAwMTA3Mzk4MDU0In0=,"{""device"":""Android"",""ecommerce"":{},""event_name"":""main"",""event_timestamp"":1593880885036129,""geo"":{""city"":""New York"",""state"":""NY""},""items"":[],""traffic_source"":""google"",""user_first_touch_timestamp"":1593880885036129,""user_id"":""UA000000107398054""}"
VUEwMDAwMDAxMDczOTI0NTg=,UA000000107392458,eyJkZXZpY2UiOiJpT1MiLCJlY29tbWVyY2UiOnt9LCJldmVudF9uYW1lIjoiYWRkX2l0ZW0iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzMDA2OTY3NTEsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg5MjI1MTMxMCwiZ2VvIjp7ImNpdHkiOiJXZXN0YnJvb2siLCJzdGF0ZSI6Ik1FIn0sIml0ZW1zIjpbeyJpdGVtX2lkIjoiTV9TVEFOX1QiLCJpdGVtX25hbWUiOiJTdGFuZGFyZCBUd2luIE1hdHRyZXNzIiwiaXRlbV9yZXZlbnVlX2luX3VzZCI6NTk1LjAsInByaWNlX2luX3VzZCI6NTk1LjAsInF1YW50aXR5IjoxfV0sInRyYWZmaWNfc291cmNlIjoiZ29vZ2xlIiwidXNlcl9maXJzdF90b3VjaF90aW1lc3RhbXAiOjE1OTM4ODAzMDA2OTY3NTEsInVzZXJfaWQiOiJVQTAwMDAwMDEwNzM5MjQ1OCJ9,"{""device"":""iOS"",""ecommerce"":{},""event_name"":""add_item"",""event_previous_timestamp"":1593880300696751,""event_timestamp"":1593880892251310,""geo"":{""city"":""Westbrook"",""state"":""ME""},""items"":[{""item_id"":""M_STAN_T"",""item_name"":""Standard Twin Mattress"",""item_revenue_in_usd"":595.0,""price_in_usd"":595.0,""quantity"":1}],""traffic_source"":""google"",""user_first_touch_timestamp"":1593880300696751,""user_id"":""UA000000107392458""}"
VUEwMDAwMDAxMDczOTU5Njg=,UA000000107395968,eyJkZXZpY2UiOiJtYWNPUyIsImVjb21tZXJjZSI6e30sImV2ZW50X25hbWUiOiJwcmVtaXVtIiwiZXZlbnRfcHJldmlvdXNfdGltZXN0YW1wIjoxNTkzODgwODYxMDMwMjQxLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODkxMjY3NzgsImdlbyI6eyJjaXR5IjoiRmlzaGVycyIsInN0YXRlIjoiSU4ifSwiaXRlbXMiOltdLCJ0cmFmZmljX3NvdXJjZSI6InlvdXR1YmUiLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg4MDY2NDY1ODc3MywidXNlcl9pZCI6IlVBMDAwMDAwMTA3Mzk1OTY4In0=,"{""device"":""macOS"",""ecommerce"":{},""event_name"":""premium"",""event_previous_timestamp"":1593880861030241,""event_timestamp"":1593880889126778,""geo"":{""city"":""Fishers"",""state"":""IN""},""items"":[],""traffic_source"":""youtube"",""user_first_touch_timestamp"":1593880664658773,""user_id"":""UA000000107395968""}"
VUEwMDAwMDAxMDczOTgwMzA=,UA000000107398030,eyJkZXZpY2UiOiJpT1MiLCJlY29tbWVyY2UiOnt9LCJldmVudF9uYW1lIjoib3JpZ2luYWwiLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODA4ODI0Mjk5ODAsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg4OTY3Njg1NywiZ2VvIjp7ImNpdHkiOiJMb21pdGEiLCJzdGF0ZSI6IkNBIn0sIml0ZW1zIjpbXSwidHJhZmZpY19zb3VyY2UiOiJmYWNlYm9vayIsInVzZXJfZmlyc3RfdG91Y2hfdGltZXN0YW1wIjoxNTkzODgwODgyNDI5OTgwLCJ1c2VyX2lkIjoiVUEwMDAwMDAxMDczOTgwMzAifQ==,"{""device"":""iOS"",""ecommerce"":{},""event_name"":""original"",""event_previous_timestamp"":1593880882429980,""event_timestamp"":1593880889676857,""geo"":{""city"":""Lomita"",""state"":""CA""},""items"":[],""traffic_source"":""facebook"",""user_first_touch_timestamp"":1593880882429980,""user_id"":""UA000000107398030""}"
VUEwMDAwMDAxMDczODIyMzM=,UA000000107382233,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6ImNjX2luZm8iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzNjQzMjEwODgsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg4NjA2NTEyNSwiZ2VvIjp7ImNpdHkiOiJOZXcgWW9yayIsInN0YXRlIjoiTlkifSwiaXRlbXMiOlt7Iml0ZW1faWQiOiJNX1NUQU5fRiIsIml0ZW1fbmFtZSI6IlN0YW5kYXJkIEZ1bGwgTWF0dHJlc3MiLCJpdGVtX3JldmVudWVfaW5fdXNkIjo5NDUuMCwicHJpY2VfaW5fdXNkIjo5NDUuMCwicXVhbnRpdHkiOjF9XSwidHJhZmZpY19zb3VyY2UiOiJpbnN0YWdyYW0iLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg3OTE5NTI2MjA0NywidXNlcl9pZCI6IlVBMDAwMDAwMTA3MzgyMjMzIn0=,"{""device"":""Android"",""ecommerce"":{},""event_name"":""cc_info"",""event_previous_timestamp"":1593880364321088,""event_timestamp"":15938808860651

3. Now, let's put it all together to create another bronze-level table named **kafka_events_bronze_decoded**. This table will store the STRING values for the **key** and **value** columns from the original **kafka_events_bronze_raw** table.

In [0]:
CREATE OR REPLACE TABLE kafka_events_bronze_decoded AS
SELECT
  cast(unbase64(key) AS STRING) AS decoded_key,
  offset,
  partition,
  timestamp,
  topic,
  cast(unbase64(value) AS STRING) AS decoded_value
FROM kafka_events_bronze_raw;


-- View the new table
SELECT *
FROM kafka_events_bronze_decoded
LIMIT 5;

decoded_key,offset,partition,timestamp,topic,decoded_value
UA000000107398054,219255030,0,1593880885085,clickstream,"{""device"":""Android"",""ecommerce"":{},""event_name"":""main"",""event_timestamp"":1593880885036129,""geo"":{""city"":""New York"",""state"":""NY""},""items"":[],""traffic_source"":""google"",""user_first_touch_timestamp"":1593880885036129,""user_id"":""UA000000107398054""}"
UA000000107392458,219255043,0,1593880892303,clickstream,"{""device"":""iOS"",""ecommerce"":{},""event_name"":""add_item"",""event_previous_timestamp"":1593880300696751,""event_timestamp"":1593880892251310,""geo"":{""city"":""Westbrook"",""state"":""ME""},""items"":[{""item_id"":""M_STAN_T"",""item_name"":""Standard Twin Mattress"",""item_revenue_in_usd"":595.0,""price_in_usd"":595.0,""quantity"":1}],""traffic_source"":""google"",""user_first_touch_timestamp"":1593880300696751,""user_id"":""UA000000107392458""}"
UA000000107395968,219255108,0,1593880889174,clickstream,"{""device"":""macOS"",""ecommerce"":{},""event_name"":""premium"",""event_previous_timestamp"":1593880861030241,""event_timestamp"":1593880889126778,""geo"":{""city"":""Fishers"",""state"":""IN""},""items"":[],""traffic_source"":""youtube"",""user_first_touch_timestamp"":1593880664658773,""user_id"":""UA000000107395968""}"
UA000000107398030,219255118,0,1593880889725,clickstream,"{""device"":""iOS"",""ecommerce"":{},""event_name"":""original"",""event_previous_timestamp"":1593880882429980,""event_timestamp"":1593880889676857,""geo"":{""city"":""Lomita"",""state"":""CA""},""items"":[],""traffic_source"":""facebook"",""user_first_touch_timestamp"":1593880882429980,""user_id"":""UA000000107398030""}"
UA000000107382233,219438025,1,1593880886106,clickstream,"{""device"":""Android"",""ecommerce"":{},""event_name"":""cc_info"",""event_previous_timestamp"":1593880364321088,""event_timestamp"":1593880886065125,""geo"":{""city"":""New York"",""state"":""NY""},""items"":[{""item_id"":""M_STAN_F"",""item_name"":""Standard Full Mattress"",""item_revenue_in_usd"":945.0,""price_in_usd"":945.0,""quantity"":1}],""traffic_source"":""instagram"",""user_first_touch_timestamp"":1593879195262047,""user_id"":""UA000000107382233""}"


## C. Working with JSON Formatted Strings in a Table

### C1. Flattening JSON String Columns

Next, we will explore how extract a column from a column containing a JSON formatted string. 


**BENEFITS**
- **Simple** - Easy to implement and store JSON as plain text.
- **Flexible** - Can hold any JSON structure without schema constraints.

**CONSIDERATIONS**
- **Performance** - STRING columns are slower when querying and processing complex data.
- **No Schema** - The lack of a defined schema for STRING columns can lead to data integrity issues.
- **Complex to Query** - Requires additional code to parse and retrieve data, which can be complex.

#### C1.1 Query JSON strings

You can extract a column from fields containing JSON strings using the syntax: `<column-name>:<extraction-path>`, where `<column-name>` is the string column name and `<extraction-path>` is the path to the field to extract. The returned results are strings. You can also do this with nested fields by using either `.` or `[]`.

This utilizes Spark SQL's built-in functionality to interact directly with nested data stored as JSON strings.

[Query JSON strings](https://docs.databricks.com/aws/en/semi-structured/json)


Example JSON string pulled from a row in the column **decoded_value**:


```
{
    "device": "iOS",
    "ecommerce": {},
    "event_name": "add_item",
    "event_previous_timestamp": 1593880300696751,
    "event_timestamp": 1593880892251310,
    "geo": {
      "city": "Westbrook", 
      "state": "ME"
      },
    "items": [
        {
            "item_id": "M_STAN_T",
            "item_name": "Standard Twin Mattress",
            "item_revenue_in_usd": 595.0,
            "price_in_usd": 595.0,
            "quantity": 1,
        }
    ],
    "traffic_source": "google",
    "user_first_touch_timestamp": 1593880300696751,
    "user_id": "UA000000107392458",
}
```

1. For example, let's extract the following values from the JSON-formatted string:
    - `decoded_value:device`
    - `decoded_value:traffic_source`
    - `decoded_value:geo`
    - `decoded_value:items`

    Run the cell and view the results. Notice that we have successfully extracted the values from the JSON formatted string.

    - **device** is a STRING

    - **traffic_source** is a STRING

    - **geo** is a STRING containing another JSON formatted string
    
    - **item** is a STRING contain an array of JSON formatted strings


In [0]:
SELECT 
  decoded_value,
  decoded_value:device,
  decoded_value:traffic_source,
  decoded_value:geo,       ----- Contains another JSON formatted string
  decoded_value:items      ----- Contains a nested-array of JSON formatted strings
FROM kafka_events_bronze_decoded
LIMIT 5;

decoded_value,device,traffic_source,geo,items
"{""device"":""Android"",""ecommerce"":{},""event_name"":""main"",""event_timestamp"":1593880885036129,""geo"":{""city"":""New York"",""state"":""NY""},""items"":[],""traffic_source"":""google"",""user_first_touch_timestamp"":1593880885036129,""user_id"":""UA000000107398054""}",Android,google,"{""city"":""New York"",""state"":""NY""}",[]
"{""device"":""iOS"",""ecommerce"":{},""event_name"":""add_item"",""event_previous_timestamp"":1593880300696751,""event_timestamp"":1593880892251310,""geo"":{""city"":""Westbrook"",""state"":""ME""},""items"":[{""item_id"":""M_STAN_T"",""item_name"":""Standard Twin Mattress"",""item_revenue_in_usd"":595.0,""price_in_usd"":595.0,""quantity"":1}],""traffic_source"":""google"",""user_first_touch_timestamp"":1593880300696751,""user_id"":""UA000000107392458""}",iOS,google,"{""city"":""Westbrook"",""state"":""ME""}","[{""item_id"":""M_STAN_T"",""item_name"":""Standard Twin Mattress"",""item_revenue_in_usd"":595.0,""price_in_usd"":595.0,""quantity"":1}]"
"{""device"":""macOS"",""ecommerce"":{},""event_name"":""premium"",""event_previous_timestamp"":1593880861030241,""event_timestamp"":1593880889126778,""geo"":{""city"":""Fishers"",""state"":""IN""},""items"":[],""traffic_source"":""youtube"",""user_first_touch_timestamp"":1593880664658773,""user_id"":""UA000000107395968""}",macOS,youtube,"{""city"":""Fishers"",""state"":""IN""}",[]
"{""device"":""iOS"",""ecommerce"":{},""event_name"":""original"",""event_previous_timestamp"":1593880882429980,""event_timestamp"":1593880889676857,""geo"":{""city"":""Lomita"",""state"":""CA""},""items"":[],""traffic_source"":""facebook"",""user_first_touch_timestamp"":1593880882429980,""user_id"":""UA000000107398030""}",iOS,facebook,"{""city"":""Lomita"",""state"":""CA""}",[]
"{""device"":""Android"",""ecommerce"":{},""event_name"":""cc_info"",""event_previous_timestamp"":1593880364321088,""event_timestamp"":1593880886065125,""geo"":{""city"":""New York"",""state"":""NY""},""items"":[{""item_id"":""M_STAN_F"",""item_name"":""Standard Full Mattress"",""item_revenue_in_usd"":945.0,""price_in_usd"":945.0,""quantity"":1}],""traffic_source"":""instagram"",""user_first_touch_timestamp"":1593879195262047,""user_id"":""UA000000107382233""}",Android,instagram,"{""city"":""New York"",""state"":""NY""}","[{""item_id"":""M_STAN_F"",""item_name"":""Standard Full Mattress"",""item_revenue_in_usd"":945.0,""price_in_usd"":945.0,""quantity"":1}]"


2. We can then begin to parse out the necessary JSON formatted string values to create another bronze table to flatten the JSON formatted string column for downstream processing.

In [0]:
CREATE OR REPLACE TABLE kafka_events_bronze_string_flattened AS
SELECT
  decoded_key,
  offset,
  partition,
  timestamp,
  topic,
  decoded_value:device,
  decoded_value:traffic_source,
  decoded_value:geo,       ----- Contains another JSON formatted string
  decoded_value:items      ----- Contains a nested-array of JSON formatted strings
FROM kafka_events_bronze_decoded;


-- Display the table
SELECT *
FROM kafka_events_bronze_string_flattened;

decoded_key,offset,partition,timestamp,topic,device,traffic_source,geo,items
UA000000107398054,219255030,0,1593880885085,clickstream,Android,google,"{""city"":""New York"",""state"":""NY""}",[]
UA000000107392458,219255043,0,1593880892303,clickstream,iOS,google,"{""city"":""Westbrook"",""state"":""ME""}","[{""item_id"":""M_STAN_T"",""item_name"":""Standard Twin Mattress"",""item_revenue_in_usd"":595.0,""price_in_usd"":595.0,""quantity"":1}]"
UA000000107395968,219255108,0,1593880889174,clickstream,macOS,youtube,"{""city"":""Fishers"",""state"":""IN""}",[]
UA000000107398030,219255118,0,1593880889725,clickstream,iOS,facebook,"{""city"":""Lomita"",""state"":""CA""}",[]
UA000000107382233,219438025,1,1593880886106,clickstream,Android,instagram,"{""city"":""New York"",""state"":""NY""}","[{""item_id"":""M_STAN_F"",""item_name"":""Standard Full Mattress"",""item_revenue_in_usd"":945.0,""price_in_usd"":945.0,""quantity"":1}]"
UA000000107382233,219438069,1,1593880886106,clickstream,Android,instagram,"{""city"":""New York"",""state"":""NY""}","[{""item_id"":""M_STAN_F"",""item_name"":""Standard Full Mattress"",""item_revenue_in_usd"":945.0,""price_in_usd"":945.0,""quantity"":1}]"
UA000000107398037,219438089,1,1593880887640,clickstream,Android,facebook,"{""city"":""Vernon"",""state"":""TX""}",[]
UA000000107398159,219438114,1,1593880894803,clickstream,macOS,youtube,"{""city"":""Lakewood"",""state"":""CO""}",[]
UA000000107376467,219438126,1,1593880888445,clickstream,Windows,facebook,"{""city"":""Denver"",""state"":""CO""}","[{""item_id"":""M_PREM_K"",""item_name"":""Premium King Mattress"",""item_revenue_in_usd"":1995.0,""price_in_usd"":1995.0,""quantity"":1},{""item_id"":""M_STAN_Q"",""item_name"":""Standard Queen Mattress"",""item_revenue_in_usd"":2090.0,""price_in_usd"":1045.0,""quantity"":2}]"
UA000000107398037,219438135,1,1593880887640,clickstream,Android,facebook,"{""city"":""Vernon"",""state"":""TX""}",[]


### C2. Flattening JSON Formatting Strings via STRUCT Conversion

Similar to the previous section, we will discuss how to flatten our JSON STRING column **decoded_value** using a STRUCT column.

#### Benefits and Considerations of STRUCT Columns

**Benefits**
- **Schema Enforcement** – STRUCT columns define and enforce a schema, helping maintain data integrity.
- **Improved Performance** – STRUCTs are generally more efficient for querying and processing than plain strings.

**Considerations**
- **Schema Enforcement** – Because the schema is enforced, issues can arise if the JSON structure changes over time.
- **Reduced Flexibility** – The data must consistently match the defined schema, leaving less room for structural variation.

#### C2.1 Converting a JSON STRING to a STRUCT Column
To convert a JSON-formatted STRING column to a STRUCT column, you will need to derive the schema of the JSON-formatted string and then parse each row into a STRUCT type.

We can do this in two steps.
  1. Get the STRUCT type of the JSON formatted string.
  2. Apply the STRUCT to the JSON formatted string column.

**NOTE:** We have already copied and pasted the correct values for you as a part of this demonstration. The subsequent cell below is a copy and paste of the output of the single row that appears when running the next cell. 


1. Determine the derived schema using the [**`schema_of_json()`**](https://docs.databricks.com/en/sql/language-manual/functions/schema_of_json.html) function, which returns the schema inferred from a JSON-formatted string.

    Run the cell and view the results. Notice that the output displays the structure of the JSON string.

In [0]:
SELECT schema_of_json('{"device":"Linux","ecommerce":{"purchase_revenue_in_usd":1075.5,"total_item_quantity":1,"unique_items":1},"event_name":"finalize","event_previous_timestamp":1593879231210816,"event_timestamp":1593879335779563,"geo":{"city":"Houston","state":"TX"},"items":[{"coupon":"NEWBED10","item_id":"M_STAN_K","item_name":"Standard King Mattress","item_revenue_in_usd":1075.5,"price_in_usd":1195.0,"quantity":1}],"traffic_source":"email","user_first_touch_timestamp":1593454417513109,"user_id":"UA000000106116176"}')
AS schema

schema
"STRUCT, event_name: STRING, event_previous_timestamp: BIGINT, event_timestamp: BIGINT, geo: STRUCT, items: ARRAY>, traffic_source: STRING, user_first_touch_timestamp: BIGINT, user_id: STRING>"


2. Copy and paste the output from `schema_of_json` into the [**`from_json()`**](https://docs.databricks.com/en/sql/language-manual/functions/from_json.html) function. This function parses a column containing a JSON-formatted string into a STRUCT type using the specified schema, and creates a new table named **kafka_events_bronze_struct**.

    Run the cell and view the results. Notice that the **value** column has been transformed into a nested STRUCT that includes scalar fields, nested structs, and an array of structs.

In [0]:
CREATE OR REPLACE TABLE kafka_events_bronze_struct AS
SELECT 
  * EXCEPT (decoded_value),
  from_json(
      decoded_value,    -- JSON formatted string column
      'STRUCT<device: STRING, ecommerce: STRUCT<purchase_revenue_in_usd: DOUBLE, total_item_quantity: BIGINT, unique_items: BIGINT>, event_name: STRING, event_previous_timestamp: BIGINT, event_timestamp: BIGINT, geo: STRUCT<city: STRING, state: STRING>, items: ARRAY<STRUCT<coupon: STRING, item_id: STRING, item_name: STRING, item_revenue_in_usd: DOUBLE, price_in_usd: DOUBLE, quantity: BIGINT>>, traffic_source: STRING, user_first_touch_timestamp: BIGINT, user_id: STRING>') AS value
FROM kafka_events_bronze_decoded;


-- View the new table.
SELECT *
FROM kafka_events_bronze_struct
LIMIT 5;

decoded_key,offset,partition,timestamp,topic,value
UA000000107398054,219255030,0,1593880885085,clickstream,"List(Android, List(null, null, null), main, null, 1593880885036129, List(New York, NY), List(), google, 1593880885036129, UA000000107398054)"
UA000000107392458,219255043,0,1593880892303,clickstream,"List(iOS, List(null, null, null), add_item, 1593880300696751, 1593880892251310, List(Westbrook, ME), List(List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1)), google, 1593880300696751, UA000000107392458)"
UA000000107395968,219255108,0,1593880889174,clickstream,"List(macOS, List(null, null, null), premium, 1593880861030241, 1593880889126778, List(Fishers, IN), List(), youtube, 1593880664658773, UA000000107395968)"
UA000000107398030,219255118,0,1593880889725,clickstream,"List(iOS, List(null, null, null), original, 1593880882429980, 1593880889676857, List(Lomita, CA), List(), facebook, 1593880882429980, UA000000107398030)"
UA000000107382233,219438025,1,1593880886106,clickstream,"List(Android, List(null, null, null), cc_info, 1593880364321088, 1593880886065125, List(New York, NY), List(List(null, M_STAN_F, Standard Full Mattress, 945.0, 945.0, 1)), instagram, 1593879195262047, UA000000107382233)"


#### C2.2 Extract fields, nested fields, and nested arrays from STRUCT columns

We can query the STRUCT column using `value.device` or `value.ecommerce` in our SELECT statement.

1. Using this syntax, we can obtain values from the **value** struct column. Run the cell and view the results. Notice the following:

    - We obtained values from the STRUCT column for **device** and **city**
    
    - The **items** column contains an ARRAY of STRUCTS. The number of elements in the array varies.

In [0]:
SELECT 
  decoded_key,
  value.device as device,  -- <----- Field
  value.geo.city as city,  -- <----- Nested-field from geo field
  value.items as items,
  array_size(items) AS number_elements_in_array -- <----- Count the number of elements in the array column items
FROM kafka_events_bronze_struct
ORDER BY number_elements_in_array DESC;

decoded_key,device,city,items,number_elements_in_array
UA000000107380131,Windows,New York,"List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1), List(null, P_FOAM_S, Standard Foam Pillow, 59.0, 59.0, 1), List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1))",3
UA000000107342625,Linux,Bartlett,"List(List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1), List(null, M_STAN_Q, Standard Queen Mattress, 1045.0, 1045.0, 1))",2
UA000000107376467,Windows,Denver,"List(List(null, M_PREM_K, Premium King Mattress, 1995.0, 1995.0, 1), List(null, M_STAN_Q, Standard Queen Mattress, 2090.0, 1045.0, 2))",2
UA000000107352551,Windows,New Orleans,"List(List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1), List(null, P_DOWN_K, King Down Pillow, 159.0, 159.0, 1))",2
UA000000107376467,Windows,Denver,"List(List(null, M_PREM_K, Premium King Mattress, 1995.0, 1995.0, 1), List(null, M_STAN_Q, Standard Queen Mattress, 2090.0, 1045.0, 2))",2
UA000000107387142,Android,Lucas,"List(List(null, M_PREM_Q, Premium Queen Mattress, 1795.0, 1795.0, 1), List(null, P_FOAM_K, King Foam Pillow, 79.0, 79.0, 1))",2
UA000000107376467,Windows,Denver,"List(List(null, M_PREM_K, Premium King Mattress, 1995.0, 1995.0, 1), List(null, M_STAN_Q, Standard Queen Mattress, 2090.0, 1045.0, 2))",2
UA000000107377542,Chrome OS,Evansville,"List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1), List(null, P_FOAM_K, King Foam Pillow, 79.0, 79.0, 1))",2
UA000000107352551,Windows,New Orleans,"List(List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1), List(null, P_DOWN_K, King Down Pillow, 159.0, 159.0, 1))",2
UA000000106459577,Linux,Huntington Park,"List(List(NEWBED10, M_STAN_Q, Standard Queen Mattress, 940.5, 1045.0, 1), List(NEWBED10, P_DOWN_S, Standard Down Pillow, 107.10000000000001, 119.0, 1))",2


#### C2.3 Explode Arrays

Exploding an array transforms each element of an array column into a separate row, effectively flattening the array. There are a few things to keep in mind when using this function. 

1. It returns a set of rows composed of the elements of the array or the keys and values of the map.

1. If the array is `NULL` no rows are produced. To return a single row with `NULL`s for the array or map values use the [`explode_outer()`](https://docs.databricks.com/gcp/en/sql/language-manual/functions/explode_outer) function.

1. Run the cell to see how the ARRAY of values in the `value.items` explodes the array into one row for each element in the array.

In [0]:
CREATE OR REPLACE TABLE bronze_explode_array AS
SELECT
  decoded_key,
  array_size(value.items) AS number_elements_in_array,
  explode(value.items) AS item_in_array,
  value.items
FROM kafka_events_bronze_struct
ORDER BY number_elements_in_array DESC;


-- Display table
SELECT *
FROM bronze_explode_array;

decoded_key,number_elements_in_array,item_in_array,items
UA000000107380131,3,"List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1)","List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1), List(null, P_FOAM_S, Standard Foam Pillow, 59.0, 59.0, 1), List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1))"
UA000000107380131,3,"List(null, P_FOAM_S, Standard Foam Pillow, 59.0, 59.0, 1)","List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1), List(null, P_FOAM_S, Standard Foam Pillow, 59.0, 59.0, 1), List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1))"
UA000000107380131,3,"List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1)","List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1), List(null, P_FOAM_S, Standard Foam Pillow, 59.0, 59.0, 1), List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1))"
UA000000107376467,2,"List(null, M_PREM_K, Premium King Mattress, 1995.0, 1995.0, 1)","List(List(null, M_PREM_K, Premium King Mattress, 1995.0, 1995.0, 1), List(null, M_STAN_Q, Standard Queen Mattress, 2090.0, 1045.0, 2))"
UA000000107376467,2,"List(null, M_STAN_Q, Standard Queen Mattress, 2090.0, 1045.0, 2)","List(List(null, M_PREM_K, Premium King Mattress, 1995.0, 1995.0, 1), List(null, M_STAN_Q, Standard Queen Mattress, 2090.0, 1045.0, 2))"
UA000000107376467,2,"List(null, M_PREM_K, Premium King Mattress, 1995.0, 1995.0, 1)","List(List(null, M_PREM_K, Premium King Mattress, 1995.0, 1995.0, 1), List(null, M_STAN_Q, Standard Queen Mattress, 2090.0, 1045.0, 2))"
UA000000107376467,2,"List(null, M_STAN_Q, Standard Queen Mattress, 2090.0, 1045.0, 2)","List(List(null, M_PREM_K, Premium King Mattress, 1995.0, 1995.0, 1), List(null, M_STAN_Q, Standard Queen Mattress, 2090.0, 1045.0, 2))"
UA000000107376053,2,"List(null, M_STAN_F, Standard Full Mattress, 945.0, 945.0, 1)","List(List(null, M_STAN_F, Standard Full Mattress, 945.0, 945.0, 1), List(null, P_FOAM_S, Standard Foam Pillow, 59.0, 59.0, 1))"
UA000000107376053,2,"List(null, P_FOAM_S, Standard Foam Pillow, 59.0, 59.0, 1)","List(List(null, M_STAN_F, Standard Full Mattress, 945.0, 945.0, 1), List(null, P_FOAM_S, Standard Foam Pillow, 59.0, 59.0, 1))"
UA000000107380131,2,"List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1)","List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1), List(null, P_FOAM_S, Standard Foam Pillow, 59.0, 59.0, 1))"


## D. Working with a VARIANT Column (Public Preview)

#### VARIANT Column Benefits and Considerations:

**BENEFITS**
- **Open** - Fully open-sourced, no proprietary data lock-in.
- **Flexible** - No strict schema. You can put any type of semi-structured data into VARIANT.
- **Performant** - Improved performance over existing methods.

**CONSIDERATIONS**
- Currently in public preview as of 2025 Q2.
- [Variant support in Delta Lake](https://docs.databricks.com/aws/en/delta/variant)

**RESOURCES**:
- [Introducing the Open Variant Data Type in Delta Lake and Apache Spark](https://www.databricks.com/blog/introducing-open-variant-data-type-delta-lake-and-apache-spark)
- [Say goodbye to messy JSON headaches with VARIANT](https://www.youtube.com/watch?v=fWdxF7nL3YI)
- [Variant Data Type - Making Semi-Structured Data Fast and Simple](https://www.youtube.com/watch?v=jtjOfggD4YY)


**NOTE:** Variant data type will not work on Serverless Version 1.

1. View the **kafka_events_bronze_decoded** table. Confirm the **decoded_value** column contains a JSON formatted string.

In [0]:
SELECT *
FROM kafka_events_bronze_decoded
LIMIT 5;

2. Use the [`parse_json`](https://docs.databricks.com/aws/en/sql/language-manual/functions/parse_json) function to returns a VARIANT value from the JSON formatted string.

    Run the cell and view the results. Notice that the **json_variant_value** column is of type VARIANT.

In [0]:
CREATE OR REPLACE TABLE kafka_events_bronze_variant AS
SELECT
  decoded_key,
  offset,
  partition,
  timestamp,
  topic,
  parse_json(decoded_value) AS json_variant_value   -- Convert the decoded_value column to a variant data type
FROM kafka_events_bronze_decoded;

-- View the table
SELECT *
FROM kafka_events_bronze_variant
LIMIT 5;

3. You can parse the VARIANT data type column using `:` to create your desired table.

    [VARIANT type](https://docs.databricks.com/aws/en/sql/language-manual/data-types/variant-type)

In [0]:
SELECT
  json_variant_value,
  json_variant_value:device :: STRING,  -- Obtain the value of device and cast to a string
  json_variant_value:items
FROM kafka_events_bronze_variant
LIMIT 10;


&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>